In [360]:
import nltk
import numpy as np
from pyvi import ViTokenizer

In [361]:
# Tập data chatbot

In [362]:
file_tranning = ['./dữ liệu chatbot - dự án cuối kì/bạn bè.txt',
'./dữ liệu chatbot - dự án cuối kì/các câu hỏi phức tạp.txt',
'./dữ liệu chatbot - dự án cuối kì/đất nước.txt',
'./dữ liệu chatbot - dự án cuối kì/địa chỉ.txt',
'./dữ liệu chatbot - dự án cuối kì/du lịch.txt',
'./dữ liệu chatbot - dự án cuối kì/gia đình.txt',
'./dữ liệu chatbot - dự án cuối kì/giải trí.txt',
'./dữ liệu chatbot - dự án cuối kì/học tập.txt',
'./dữ liệu chatbot - dự án cuối kì/nghề nghiệp.txt',
'./dữ liệu chatbot - dự án cuối kì/nghỉ lễ.txt',
'./dữ liệu chatbot - dự án cuối kì/người yêu.txt',
'./dữ liệu chatbot - dự án cuối kì/robot.txt',
'./dữ liệu chatbot - dự án cuối kì/shoping.txt',
'./dữ liệu chatbot - dự án cuối kì/shoping.txt',
'./dữ liệu chatbot - dự án cuối kì/tán gẫu.txt',
'./dữ liệu chatbot - dự án cuối kì/tdtu.txt',
'./dữ liệu chatbot - dự án cuối kì/thông tin cá nhân.txt',
'./dữ liệu chatbot - dự án cuối kì/trò chuyện về đi ăn.txt']

In [363]:
question_train = []
answer_train = []
for i in range(len(file_tranning)):
    with open(file_tranning[i], encoding='UTF-8') as f:
        train_lines = f.readlines()
        for line in train_lines:
            tmp = line.split("__eou__")
            question_train.append(tmp[0].strip()) # strip(): Loại bỏ whitespace đầu cuối string
            answer_train.append(tmp[1].strip())

In [364]:
# print(question_train)

In [365]:
print((question_train[0]))
print((answer_train[0]))

Thích đánh lộn không?
Ngon nhà vô


In [366]:
# Word Segmentation 
for i in range(len(question_train)):
    question_train[i] = ViTokenizer.tokenize(question_train[i])
    answer_train[i] = ViTokenizer.tokenize(answer_train[i])

In [367]:
print(question_train[0])
print(answer_train[0])

Thích đánh_lộn không ?
Ngon nhà_vô


In [368]:
# Tập vietnamese-stopwords-dash.txt
# file_stopwords = './vietnamese-stopwords-dash.txt'

# stopwords = []

# with open(file_stopwords, encoding='UTF-8') as f:
#     stopwords_lines = f.readlines()
#     for line in stopwords_lines:
#         stopwords.append(line.rstrip("\n"))

In [369]:
# print(stopwords)

In [370]:
import string
my_punctuation = (string.punctuation).replace("_","") # remove _ in my_punctuation

In [371]:
print(my_punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^`{|}~


In [372]:
def text_process(mess):
#     STOPWORDS = stopwords
    nopunc = [char for char in mess if char not in my_punctuation]
#     nopunc = ''.join(nopunc)
    return  ''.join(nopunc)
#     return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [373]:
# Bỏ dấu câu
for i in range(len(question_train)):
    question_train[i] = text_process(question_train[i])
    answer_train[i] = text_process(answer_train[i])

In [374]:
print(question_train[0])
print(answer_train[0])

Thích đánh_lộn không 
Ngon nhà_vô


In [375]:
from nltk import word_tokenize
sents=[]
for i in range(len(question_train)):
    tokens = word_tokenize(question_train[i])
    sents.append(tokens)

In [376]:
# print(sents)

In [377]:
from gensim.models import Word2Vec
modelw2v = Word2Vec(sentences=sents, size=100, window=5, sg=1, min_count=2)

In [378]:
# chuyển 1 câu thành 1 vector
# vector đại diện cho câu theo phương pháp trung bình
np.seterr(divide='ignore', invalid='ignore')

def convert2vec(words):
    sum_ = np.array([0]*100)
    for word in words:
        if(word not in modelw2v.wv):
            continue
        vec = modelw2v.wv[word]
        sum_ = sum_ + vec
    return sum_ / len(words) # Lấy vector của tất cả từ chia trung bình

In [379]:
# print(1 - spatial.distance.cosine(convert2vec(ViTokenizer.tokenize("Thích đánh lộn không?")), convert2vec(question_train[0])))

In [380]:
from scipy import spatial

In [381]:
def get_answer(question):
    question = ViTokenizer.tokenize(question)
    question = convert2vec(question)
    
    cosin_lst = []
    for i in range(len(question_train)):
        cosin_lst.append(1 - spatial.distance.cosine(question, convert2vec(question_train[i])))
#     print(cosin_lst)

    question_pred = max(cosin_lst)
    index = cosin_lst.index(question_pred)
    return answer_train[index]

In [382]:
question_input = "1 + 1 bằng mấy?"
answer = get_answer(question_input)

In [383]:
print("Q: ", question_input)
print("A: ", answer)

Q:  1 + 1 bằng mấy?
A:  Bằng 2  Câu hỏi này dành cho học_sinh tiểu_học 


In [386]:
while True:
    question_input = input("Q: ")
    if(question_input.strip() == "stop"):
        break
    print("A:", get_answer(question_input))


Q: hehe
A: chị học CNTT  con_gái học ngành này cũng thú_vị lắm  chị cũng có những người bạn tốt luôn giúp_đỡ nhau trong học_tập nên thấy cuộc_sống đại_học khá là thú_vị  nếu em thích học toán và đam_mê công_nghệ  đam_mê tìm_kiếm cái mới thì thử tìm_hiểu về CNTT nhé 
Q: TDTU
A: Có gì đâu mà buồn  kiếm đứa khác thui  lấy đó làm động_lực phấn_đấu đi
Q: bạn là ai ?
A: hiện_tại mình học ngành khoa_học máy_tính
Q: nhà bạn ở đâu?
A: Q7
Q: bạn tên gì?
A: mình với người_yêu hay đi ăn rồi đi xem phim hay đi ra các điểm mà tụi mình thích
Q: thua
A: rồi ông
Q: thế giới có bao nhiêu đại dương?
A: Có  quê mình đi xe_máy khoảng 1 tiếng
Q: hentai
A: ok để sắp_xếp
Q: bạn có người yêu chưa?
A: mình đã làm gì có người_yêu  mình còn đang sợ ế đây này
Q: stop


In [137]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer()
# X_tfidf = vectorizer.fit_transform(question_train)
# print(X_tfidf.shape)

In [136]:
# print(vectorizer.get_feature_names())